```{contents}
```
## **Reviewer Agent in LangGraph**

A **Reviewer Agent** in LangGraph is a specialized agent responsible for **evaluating, critiquing, validating, and improving** the outputs produced by other agents or nodes before those outputs are accepted as final.
It introduces **quality control, safety, and correctness guarantees** into LLM workflows.

---

### **1. Motivation and Intuition**

LLMs generate fluent text but are:

* Probabilistic
* Prone to hallucination
* Sensitive to prompt errors

A Reviewer Agent acts as a **second, independent reasoning process** that reduces failure.

| Without Reviewer  | With Reviewer    |
| ----------------- | ---------------- |
| Faster            | Safer            |
| Cheaper           | More reliable    |
| Higher error rate | Lower error rate |
| Hard to audit     | Fully auditable  |

---

### **2. Conceptual Role in the Graph**

```
User → Generator Agent → Reviewer Agent → Final Output
                       ↑         |
                       └── Revision Loop ──┘
```

The Reviewer either **approves**, **requests revision**, or **rejects**.

---

### **3. Responsibilities of a Reviewer Agent**

| Function               | Description                              |
| ---------------------- | ---------------------------------------- |
| Correctness checking   | Validate factual and logical consistency |
| Constraint enforcement | Enforce format, style, and policy rules  |
| Safety validation      | Filter unsafe or restricted content      |
| Completeness           | Ensure task requirements are satisfied   |
| Optimization           | Improve clarity and structure            |
| Risk assessment        | Identify uncertainty or errors           |

---

### **4. Implementation in LangGraph**

#### **State Schema**

```python
class State(TypedDict):
    draft: str
    feedback: str
    approved: bool
```

---

#### **Generator Node**

```python
def generate(state):
    return {"draft": llm.invoke("Write explanation of transformers")}
```

---

#### **Reviewer Node**

```python
def review(state):
    review_prompt = f"""
    Review the following text for correctness and clarity.
    Reply with APPROVED or REVISE and provide feedback.

    Text:
    {state['draft']}
    """
    result = llm.invoke(review_prompt)

    if result.startswith("APPROVED"):
        return {"approved": True}
    else:
        return {"approved": False, "feedback": result}
```

---

#### **Revision Node**

```python
def revise(state):
    revision_prompt = f"""
    Improve the text using the following feedback:
    {state['feedback']}
    """
    return {"draft": llm.invoke(revision_prompt)}
```

---

#### **Graph Construction**

```python
builder.add_node("generate", generate)
builder.add_node("review", review)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges("review", lambda s: "revise" if not s["approved"] else END, {
    "revise": "revise",
    END: END
})

builder.add_edge("revise", "review")
```

This forms a **self-correcting cycle**.

---

### **5. Reviewer Patterns**

| Pattern              | Description                   |
| -------------------- | ----------------------------- |
| Single-pass review   | One validation only           |
| Iterative refinement | Loop until approved           |
| Multi-reviewer       | Multiple reviewers vote       |
| Hierarchical review  | Junior → Senior → Final       |
| Risk-based review    | Review only high-risk outputs |

---

### **6. Production Applications**

| Domain   | Use                      |
| -------- | ------------------------ |
| Legal    | Contract review          |
| Medical  | Clinical note validation |
| Finance  | Report verification      |
| Research | Fact checking            |
| Software | Code review              |

---

### **7. Reliability Guarantees**

| Mechanism         | Benefit                 |
| ----------------- | ----------------------- |
| Dual reasoning    | Error reduction         |
| Explicit feedback | Transparent improvement |
| Audit trail       | Compliance              |
| Human-in-the-loop | Final safety net        |

---

### **8. Mental Model**

A Reviewer Agent is a **control system**:

> **Generate → Evaluate → Correct → Validate**

It converts stochastic LLM output into **deterministic, production-grade results**.


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

class State(TypedDict):
    draft: str
    feedback: str
    approved: bool

def generate(state):
    text = llm.invoke("Explain attention mechanism in transformers in 3 sentences.").content
    return {"draft": text}

def review(state):
    prompt = f"""
    Review the following text for correctness and clarity.
    If acceptable reply: APPROVED
    Else reply: REVISE: <feedback>

    Text:
    {state['draft']}
    """
    result = llm.invoke(prompt).content

    if result.startswith("APPROVED"):
        return {"approved": True, "feedback": ""}
    else:
        return {"approved": False, "feedback": result}

def revise(state):
    prompt = f"""
    Improve the text using this feedback:
    {state['feedback']}
    """
    improved = llm.invoke(prompt).content
    return {"draft": improved}

builder = StateGraph(State)
builder.add_node("generate", generate)
builder.add_node("review", review)
builder.add_node("revise", revise)

builder.set_entry_point("generate")
builder.add_edge("generate", "review")

builder.add_conditional_edges(
    "review",
    lambda s: END if s["approved"] else "revise",
    {END: END, "revise": "revise"}
)

builder.add_edge("revise", "review")

graph = builder.compile()

result = graph.invoke({})
print("\nFINAL OUTPUT:\n")
print(result["draft"])



FINAL OUTPUT:

The attention mechanism in transformers allows the model to weigh the importance of different words in a sequence when generating an output, enabling it to focus on relevant context. It computes attention scores using a scaled dot-product approach, where queries, keys, and values are derived from the input embeddings, allowing the model to capture relationships between words regardless of their distance in the sequence. This mechanism enhances the model's ability to understand context and dependencies, leading to improved performance in tasks like translation and text generation.
